# **NLP com Flair**
Este tutorial é baseado no livro "Natural Language Processing with Flair".

# 6 - Modelos de Classificação

No notebook anterior treinamos um modelo de word embedding. O word embedding é útil em tarefas como NER e PoS, pois
nosso objetivo é justamente taggear palavras (tokens). Isso não é o caso no problema de classificação.

Por exemplo, suponho que temos um texto onde um juiz dá uma sentença. O objetivo é então criar um
modelo que identifica se a decisão foi favorável ou desfavorável. Esse tipo de coisa pode ser feita
usando ferramentas como Regex. Porém, esse tipo método é menos robusto caso o texto não seja tão padronizado.
No caso dos textos de sentenças, pode ser que o juiz sempre use algo como "julgo a sentença procedente". Porém, podem
haver variações, como "julgo a sentença à favor do réu", ou alguma varição não prevista, até mesmo um erro ortográfico.

A forma de lidar com esse tipo de problema envolve criar um embedding para a sentença/documento
como um todo, e não para cada palavra. Assim, queremos avaliar se a sentença
indica decisão positiva ou negativa.

Já falamos brevemente de document embeddings no primeiro notebook.
Um tipo bem simples é o chamado `DocumentPoolEmbeddings`, que nada mais é que uma média
dos word embeddings.

Outro é o `DocumentRNNEmbeddings`, que usa redes RNN. Porém, vamos focar no
`TransformerDocumentEmbeddings`, pois estes são os que costumam apresentar melhor desempenho.

## Importando TransformerDocumentEmbedding

Utilizaremos o modelo e dados do [legalnlp](https://github.com/felipemaiapolo/legalnlp). Baixe os dados desse repositório, que se encontram em `demo/data_base.csv`.

Vamos começar importando o embedding. Vamos usar o modelo BERTikal, que foi treinado em dados jurídicos.


In [1]:
from legalnlp.clean_functions import *
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence

document_embeddings = TransformerDocumentEmbeddings('./models/BERTikal/')

/home/davibarreira/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Criando Corpus

In [2]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np


df = pd.read_csv('./data/data_base.csv')
df.drop(columns='Unnamed: 0', inplace=True)
df.dropna(inplace=True)
df = shuffle(df, random_state=7).reset_index(drop=True)

In [3]:
n_val   = int(int(df.shape[0]*0.7)* 0.1)
n_train = int(df.shape[0]*0.7-n_val)
n_test  = df.shape[0] - n_val - n_train

train = df.iloc[0:n_train]
dev   = df.iloc[n_train: n_train + n_val]
test  = df.iloc[n_train + n_val:]

In [4]:
train.to_csv('./data/train.csv',index=False, sep='\t')
test.to_csv('./data/test.csv',index=False, sep='\t')
dev.to_csv('./data/dev.csv',index=False, sep='\t')

In [5]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = './data'

# column format indicating which columns hold the text and label(s)
columns = {0: 'text', 1: 'label'}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it

lbl_type = 'label'
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         columns,
                                         skip_header=True,
                                         delimiter='\t',
                                         label_type=lbl_type)

2022-10-21 15:34:02,400 Reading data from data
2022-10-21 15:34:02,401 Train: data/train.csv
2022-10-21 15:34:02,402 Dev: data/dev.csv
2022-10-21 15:34:02,403 Test: data/test.csv


In [6]:
label_dict = corpus.make_label_dictionary(label_type=lbl_type)

2022-10-21 15:34:02,876 Computing label dictionary. Progress:


4063it [00:01, 3915.58it/s]

2022-10-21 15:34:03,918 Dictionary created for label 'label' with 4 values: H:Arquivado (seen 1905 times), H:Ativo (seen 1844 times), H:Suspenso (seen 314 times)


In [7]:
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


classifier = TextClassifier(document_embeddings,
                            label_dictionary=label_dict,
                            label_type=lbl_type)

trainer = ModelTrainer(classifier, corpus)
trainer.fine_tune('classifier',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=2)

2022-10-21 15:34:04,788 ----------------------------------------------------------------------------------------------------
2022-10-21 15:34:04,791 Model: "TextClassifier(
  (decoder): Linear(in_features=768, out_features=4, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): CrossEntropyLoss()
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(29794, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query):

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 3.95 GiB total capacity; 3.18 GiB already allocated; 10.19 MiB free; 3.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF